In [1]:
#TO DO

# - Create correct directories in get_y
# - Do I want to make the padding value 0 so i ignore all rows with 0??
# why do I get loss nan??
# I have nans in my X_train /X_test
# should i use a different optimizer for my compiler?? maybe rmsprop

In [1]:
import pandas as pd
import os
import csv
import numpy as np
from tensorflow.keras import Sequential, layers
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing.sequence import pad_sequences
import joblib

In [2]:
ROOT_DATA_DIR = '../raw_data/Out_Feature_CSVs'
#ROOT_DATA_DIR = '../raw_data/CSV_Features'
DATA_SUB_DIRs = ["Train", "Test"]
CLS_LIST = ["Bad", "Good"]

def get_y(data_path, class_list):
    
    data_dict = {"csv_paths":[], "csv_files":[], "labels":[]}
    
    for i, clss in enumerate(class_list):
        dir_path = os.path.join(data_path, clss)
        list_csvs = [l for l in os.listdir(dir_path) if l.split(".")[-1]=="csv"]
        print(f"Found {len(list_csvs)} CSVs in {dir_path} Directory")
        data_dict["csv_files"].extend(list_csvs)
        data_dict["labels"].extend([i for k in range(len(list_csvs))])
        data_dict["csv_paths"].extend([dir_path for _ in range(len(list_csvs))])
    
    total_csvs = len(data_dict["csv_files"])
    total_labels = len(data_dict["labels"])
    print(f"Number of csvs : {total_csvs}")
    print(f"Number of Labels : {total_labels}")
    return data_dict

train_data = pd.DataFrame(get_y(data_path=os.path.join(ROOT_DATA_DIR, DATA_SUB_DIRs[0]),
                               class_list=CLS_LIST))


test_data = pd.DataFrame(get_y(data_path=os.path.join(ROOT_DATA_DIR, DATA_SUB_DIRs[1]),
                               class_list=CLS_LIST))

Found 40 CSVs in ../raw_data/Out_Feature_CSVs/Train/Bad Directory
Found 40 CSVs in ../raw_data/Out_Feature_CSVs/Train/Good Directory
Number of csvs : 80
Number of Labels : 80
Found 10 CSVs in ../raw_data/Out_Feature_CSVs/Test/Bad Directory
Found 10 CSVs in ../raw_data/Out_Feature_CSVs/Test/Good Directory
Number of csvs : 20
Number of Labels : 20


In [3]:
train_data.head()

,csv_paths,csv_files,labels
0,../raw_data/Out_Feature_CSVs/Train/Bad,68.csv,0
1,../raw_data/Out_Feature_CSVs/Train/Bad,54.csv,0
2,../raw_data/Out_Feature_CSVs/Train/Bad,41.csv,0
3,../raw_data/Out_Feature_CSVs/Train/Bad,55.csv,0
4,../raw_data/Out_Feature_CSVs/Train/Bad,69.csv,0


In [4]:
train_data['csv_paths'][0]

'../raw_data/Out_Feature_CSVs/Train/Bad'

In [5]:
y_train = train_data['labels']

In [6]:
y_test = test_data['labels']

In [7]:
df = pd.read_csv('../raw_data/Out_Feature_CSVs/Train/Bad/76.csv')
df.head()

,Unnamed: 0,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,...,feature_2038,feature_2039,feature_2040,feature_2041,feature_2042,feature_2043,feature_2044,feature_2045,feature_2046,feature_2047
0,0,0.246979,0.843562,0.071968,0.124413,0.999673,0.076203,0.108685,0.965372,2.013730,...,0.023789,0.408551,1.216399,0.726352,0.114373,0.163468,0.526327,0.188219,0.061620,0.754256
1,1,0.259899,0.924879,0.050829,0.138344,1.005638,0.037128,0.135432,0.918760,2.127180,...,0.023161,0.361161,1.214534,0.646354,0.113299,0.111966,0.553718,0.155690,0.119376,0.724960
2,2,0.383684,1.082262,0.008478,0.108694,0.948020,0.055017,0.122317,0.945775,2.014079,...,0.013676,0.239370,1.234755,0.613653,0.114273,0.079649,0.711490,0.227936,0.072618,0.738956
3,3,0.333121,1.032829,0.006871,0.066413,0.872104,0.063404,0.143408,1.065072,2.022523,...,0.014408,0.239992,1.209389,0.518542,0.246874,0.054760,0.672378,0.211177,0.033930,0.654003
4,4,0.453533,0.960070,0.006348,0.061298,1.082252,0.063146,0.083700,0.886553,1.957184,...,0.004856,0.150131,1.186936,0.612378,0.283244,0.072268,0.731791,0.258656,0.000000,0.808011


In [8]:
def csv_to_list_of_lists(csv_path):
    df = pd.read_csv(csv_path)
    df.fillna(0, inplace=True)
    list_of_lists = df.iloc[:,1:].values.tolist()
    return list_of_lists

In [9]:
def get_x(dataframe):
  list_of_csv_files = [os.path.join(row['csv_paths'], row["csv_files"]) for index, row in dataframe.iterrows()]
  X_list = [csv_to_list_of_lists(f) for f in list_of_csv_files]
  return X_list

In [10]:
#X_train is a list of lists of different list lengths
X_train = get_x(train_data)

In [11]:
#normalizing the X_train
#X_train = [[float(j)/180 for j in i] for i in X_train]

In [12]:
#pad X_train so all arrays are of the same shape
X_train_pad = pad_sequences(X_train, dtype='float32',padding='post',value=-1000)
X_train_pad.shape

(80, 176, 2048)

In [13]:
#input data must be of shape(number of sequences, number of observations per sequence, observation size)
#add a last dimension of 1
#X_train_pad = np.expand_dims(X_train_pad, 2)
#X_train_pad.shape

In [14]:
#X_test is a list of lists of different list lengths
X_test = get_x(test_data)

In [15]:
#normalizing the X_test
#X_test = [[float(j)/180 for j in i] for i in X_test]

In [16]:
#pad X_test so all arrays are of the same shape
X_test_pad = pad_sequences(X_test, dtype='float32',padding='post',value=-1000)

#input data must be of shape(number of sequences, number of observations per sequence, observation size)
#add a last dimension of 1
#X_test_pad = np.expand_dims(X_test_pad, 2)

In [17]:
X_test_pad.shape

(20, 135, 2048)

In [18]:
#LSTM RNN Model
model_LSTM = Sequential()

##change masking layer to padded amount
model_LSTM.add(layers.Masking(mask_value=-1000))

#how many units should the first layer be??
model_LSTM.add(layers.LSTM(200, activation='tanh'))
model_LSTM.add(layers.Dense(150, activation='relu'))
model_LSTM.add(layers.Dense(75, activation='relu'))
model_LSTM.add(layers.Dense(50, activation='relu'))
model_LSTM.add(layers.Dense(25, activation='relu'))
model_LSTM.add(layers.Dense(10, activation='relu'))
model_LSTM.add(layers.Dense(1, activation='sigmoid'))

model_LSTM.compile(loss='binary_crossentropy', optimizer='RMSProp', metrics=['accuracy'])

In [19]:
#Fitting the model to the train set
es = EarlyStopping(patience=5)

model_LSTM.fit(X_train_pad, y_train, 
              epochs=20, 
              batch_size=32, 
              verbose=1, 
              callbacks = [es],
              validation_split=0.2,
              shuffle=True,
              )

Epoch 1/20
2/2 [==============================] - 2s 1s/step - loss: 0.6915 - accuracy: 0.5312 - val_loss: 1.0171 - val_accuracy: 0.0000e+00
Epoch 2/20
2/2 [==============================] - 1s 373ms/step - loss: 0.6523 - accuracy: 0.6719 - val_loss: 0.6814 - val_accuracy: 0.3750
Epoch 3/20
2/2 [==============================] - 1s 363ms/step - loss: 0.6485 - accuracy: 0.6406 - val_loss: 1.1047 - val_accuracy: 0.0000e+00
Epoch 4/20
2/2 [==============================] - 1s 368ms/step - loss: 0.6081 - accuracy: 0.6875 - val_loss: 0.9341 - val_accuracy: 0.3750
Epoch 5/20
2/2 [==============================] - 1s 381ms/step - loss: 0.5658 - accuracy: 0.6875 - val_loss: 0.7829 - val_accuracy: 0.2500
Epoch 6/20
2/2 [==============================] - 1s 390ms/step - loss: 0.5555 - accuracy: 0.7188 - val_loss: 0.5260 - val_accuracy: 0.8750
Epoch 7/20
2/2 [==============================] - 1s 382ms/step - loss: 0.5351 - accuracy: 0.6875 - val_loss: 1.0522 - val_accuracy: 0.1875
Epoch 8/20
2/2 

In [20]:
import pickle

In [23]:
import joblib
joblib.dump(model_LSTM,'model.joblib')

TypeError: cannot pickle '_thread.RLock' object

In [26]:
model_LSTM.save("my_model")
model_LSTM.save_weights("weights.h5")

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: my_model/assets


In [65]:
#Evaluate the model on the test set
model_LSTM.evaluate(X_test_pad, y_test)

1/1 [==============================] - 0s 839us/step - loss: 0.6796 - accuracy: 0.6500


[0.6795881986618042, 0.6499999761581421]

In [66]:
model_LSTM.evaluate(X_train_pad, y_train)

3/3 [==============================] - 0s 101ms/step - loss: 0.6504 - accuracy: 0.6125


[0.6503555774688721, 0.612500011920929]

In [68]:
model_LSTM.predict(X_test_pad)

array([[0.27839977],
       [0.3714132 ],
       [0.33198455],
       [0.2950204 ],
       [0.31309372],
       [0.29792675],
       [0.3464046 ],
       [0.30859268],
       [0.30569553],
       [0.34702703],
       [0.56824595],
       [0.5707745 ],
       [0.26610455],
       [0.3111837 ],
       [0.34964243],
       [0.6056016 ],
       [0.32217222],
       [0.2918331 ],
       [0.37164348],
       [0.29822803]], dtype=float32)

In [69]:
test_data

,csv_paths,csv_files,labels
0,../raw_data/Out_Feature_CSVs/Test/Bad,97.csv,0
1,../raw_data/Out_Feature_CSVs/Test/Bad,96.csv,0
2,../raw_data/Out_Feature_CSVs/Test/Bad,94.csv,0
3,../raw_data/Out_Feature_CSVs/Test/Bad,95.csv,0
4,../raw_data/Out_Feature_CSVs/Test/Bad,91.csv,0
5,../raw_data/Out_Feature_CSVs/Test/Bad,92.csv,0
6,../raw_data/Out_Feature_CSVs/Test/Bad,93.csv,0
7,../raw_data/Out_Feature_CSVs/Test/Bad,100.csv,0
8,../raw_data/Out_Feature_CSVs/Test/Bad,98.csv,0
9,../raw_data/Out_Feature_CSVs/Test/Bad,99.csv,0
